# Workstation Heartbeat and Live Data

This example shows how in a separate thread from the term the HeartBeat can be queried with Python. The heartbeat is queried in a separate thread once a second. The HeartBeat represents how many milliseconds it has been since the term has received something from the Thales.

This example also receives the live data. To receive the online display data, the Zahner online display must be switched off.
To do this, the following parameter must be adapted in the file C:/FLINK/usb.ini EnableODisplay=off.

In [10]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode,ThalesRemoteScriptWrapper
import time
import threading

It is not a clean solution that the connections are global variables.
This was solved for a simpler example with global variables.

In [11]:
zenniumConnection = None
zahnerZennium = None
zenniumConnectionLiveData = None

keepThreadRunning = True

# Watch Thread
The following function is used as a thread, in which the HearBeat is queried once a second. The HeartBeat time varies, for example, if EIS is measured at low frequencies, then this time is increased.

The HeartBeat is queried once per second. A timeout of 2 seconds is used to query the HeartBeat. This ensures that the Term responds within 2 seconds, otherwise it can be assumed that the Term software has crashed.

In [12]:
def watchThreadFunction():
    global keepThreadRunning
    global zenniumConnection
    global zahnerZennium
    
    print("watch thread started")
    while keepThreadRunning:
        time.sleep(1)
        try:
            beat = zahnerZennium.getWorkstationHeartBeat(2)
        except:
            print("term error watch thread")
            keepThreadRunning = False
        else:
            print("Heartbeat: " + str(beat) + " ms")
    
    print("watch thread left")
    return

# Live Data Thread
The following function is used as a thread which receives the live data instead of the online display.

Only relevant packet types are output to the console. The relevant types are written as comments in the source code.

In [13]:
def liveDataThreadFunction():
    global keepThreadRunning
    global zenniumConnectionLiveData
    
    print("live thread started")
    while keepThreadRunning:
        try:
            data = zenniumConnectionLiveData.waitForBinaryTelegram()
            packetId = data[0]
            data = data[1:]
            '''
            Type:
            1 = Init measurement begin
            2 = Measurement end
            4 = Measurement data names
            5 = Measurement data units
            6 = ASCII data
            '''
            if packetId in [1,2,4,5,6]:
                print(data.decode("ASCII"))
        except:
            '''
            The connection to the term has an error or the socket has been closed.
            '''
            print("term error live thread")
            keepThreadRunning = False
    
    print("live thread left")
    return

# Main Program Sequence

In the main program flow, the first thing that happens is that an additional connection to the term is established with the name "Logging". The live data comes via this connection.

Then the thread is started, which receives the data.

In [ ]:
if __name__ == "__main__":
    zenniumConnectionLiveData = ThalesRemoteConnection()
    zenniumConnectionLiveData.connectToTerm("localhost", "Logging")
        
    
    liveThread = threading.Thread(target=liveDataThreadFunction)
    liveThread.start()

After the connection with the live data, the nominal connection is established, which sends the commands for measurement.

In [ ]:
    zenniumConnection = ThalesRemoteConnection()
    zenniumConnection.connectToTerm("localhost", "ScriptRemote")
        
    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
    zahnerZennium.forceThalesIntoRemoteScript()
    
    zahnerZennium.calibrateOffsets()

The watch thread uses the command interface to the Thales, so it is started after initializing this connection.

In [16]:
    watchThread = threading.Thread(target=watchThreadFunction)
    watchThread.start()
    
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setPotential(0)
    zahnerZennium.setLowerFrequencyLimit(750)
    zahnerZennium.setStartFrequency(1000)
    zahnerZennium.setUpperFrequencyLimit(1500)
    zahnerZennium.setLowerNumberOfPeriods(2)
    zahnerZennium.setLowerStepsPerDecade(2)
    zahnerZennium.setUpperNumberOfPeriods(2)
    zahnerZennium.setUpperStepsPerDecade(20)
    zahnerZennium.setScanDirection("startToMax")
    zahnerZennium.setScanStrategy("single")
    
    zahnerZennium.enablePotentiostat()
    
    
    zahnerZennium.setFrequency(1)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setNumberOfPeriods(3)
    
    print("measurement start")
    
    zahnerZennium.measureEIS()
    for i in range(20):
        zahnerZennium.getPotential()
        zahnerZennium.setPotential(0)

    print("measurement end")
        
    zahnerZennium.disablePotentiostat()

watch thread started
measurement start
Heartbeat: 359 ms
3,Impedance Spectroscopy
frequency,impedance,phase,time,significance,voltage,current,
Hz,Ohm,rad,s, ,V,A,
Heartbeat: 250 ms
Heartbeat: 766 ms
Heartbeat: 1781 ms
 1.00940e+03, 7.65470e+02,-1.55905e-02, 0.00000e+00, 9.95000e-01, 2.52615e-02,-2.01349e-06,
Heartbeat: 0 ms
Heartbeat: 954 ms
 1.11450e+03, 7.56186e+02,-1.81091e-02, 2.13700e+00, 9.94000e-01, 2.52615e-02,-2.01331e-06,
Heartbeat: 594 ms
 1.23050e+03, 7.69415e+02,-1.18808e-02, 3.49100e+00, 9.97000e-01, 2.52615e-02,-2.01360e-06,
Heartbeat: 172 ms
Heartbeat: 1172 ms
 1.35860e+03, 7.77636e+02,-8.49820e-03, 4.90950e+00, 9.96000e-01, 2.52615e-02,-2.01254e-06,
Heartbeat: 829 ms
 1.50000e+03, 7.61770e+02,-2.04173e-02, 6.25850e+00, 9.96000e-01, 2.52615e-02,-2.01279e-06,
Heartbeat: 484 ms
 1.35860e+03, 7.59320e+02,-1.62248e-02, 7.60700e+00, 9.96000e-01, 2.52615e-02,-2.01160e-06,
Heartbeat: 140 ms
Heartbeat: 1140 ms
 1.23050e+03, 7.63513e+02,-1.44088e-02, 8.95800e+00, 9.95000e-01, 2.

'OK\r'

Closing the threads and then waiting until they are closed.

In [17]:
    print("set thread kill flag")
    keepThreadRunning = False
    
    print("disconnect connections")
    zenniumConnection.disconnectFromTerm()
    zenniumConnectionLiveData.disconnectFromTerm()
    
    print("join the threads")
    liveThread.join()
    watchThread.join()
    
    print("finish")

Heartbeat: 0 ms
set thread kill flag
disconnect connections
term error live thread
live thread left
term error watch thread
watch thread left
join the threads
finish
